In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from  pyspark.sql.functions import to_date
from pyspark.sql.functions import col

conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

spark = (
    SparkSession.builder
    .appName("Desafio2_business")
    .getOrCreate()
)

In [ ]:
!aws s3 cp s3://722687989925-trusted/ ./dadosDaTrusted --recursive

In [ ]:
df = spark.read.parquet("./dadosDaTrusted/")

In [28]:
df.printSchema()

root
 |-- nome: string (nullable = true)
 |-- placa: string (nullable = true)
 |-- modelo_carro: string (nullable = true)
 |-- cor_carro: string (nullable = true)
 |-- total_multas: integer (nullable = true)
 |-- genero: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- ultima_multa: date (nullable = true)
 |-- ultima_multa_endereço: string (nullable = true)
 |-- estado: string (nullable = true)



In [40]:
norte = ["AM", 'PA', 'AC', 'RR', 'RO', 'AP', 'TO']
nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA']
sul = ['PR', 'SC', 'RS']
sudeste = ['ES', 'MG', 'RJ', 'SP']
centro_oeste = ['GO', 'MT', 'MS', 'DF']

from pyspark.sql.functions import when
from pyspark.sql.functions import col

df_transform = df.withColumn('regiao', 
                   when(df['estado'].isin(norte), "norte")
                   .when(df['estado'].isin(nordeste), "nordeste")
                   .when(df['estado'].isin(sul), "sul")
                   .when(df['estado'].isin(sudeste), "sudeste")
                   .when(df['estado'].isin(centro_oeste), "centro_oeste")
                   .otherwise(0))

df_transform = df_transform.groupBy("regiao").sum("total_multas")

In [41]:
df_transform.write.mode('overwrite').parquet('business/')

In [42]:
#uploading to business
!aws s3 cp ./business/ s3://722687989925-business/ --recursive

upload: business/_SUCCESS to s3://722687989925-business/_SUCCESS
upload: business/.part-00000-9d570de6-1e06-4e23-b380-e32e3ea4abd6-c000.snappy.parquet.crc to s3://722687989925-business/.part-00000-9d570de6-1e06-4e23-b380-e32e3ea4abd6-c000.snappy.parquet.crc
upload: business/part-00000-9d570de6-1e06-4e23-b380-e32e3ea4abd6-c000.snappy.parquet to s3://722687989925-business/part-00000-9d570de6-1e06-4e23-b380-e32e3ea4abd6-c000.snappy.parquet
upload: business/._SUCCESS.crc to s3://722687989925-business/._SUCCESS.crc


In [43]:
df_transform.printSchema()

root
 |-- regiao: string (nullable = false)
 |-- sum(total_multas): long (nullable = true)

